# Create GIF/MP4 animation for 2D horizontal plots
**Author: Jun Sasaki  Coded on 2025-01-12  Updated on 2025-04-27**<br>
Create a GIF/MP4 animation. Customization can be made by defining `post_process_func` statically (no time change) or dynamically (change with time), or `post_process_func=None` without customization. Currently, `create_mp4` does not work and converting GIF to MP4 using `convert_gif_to_mp4`. 

```Python
def post_process_func(ax, da=None, time=None):
    """
    Example of post_process_func for customizing plot (e.g., add text or markers)
    
    Parameters:
    - ax: matplotlib axis.
    - da: DataArray (optional and used for dynamic customizing).
    - time: Frame time (optional and used for dynamic customizing).
    """
```

In [ ]:
from xfvcom import FvcomDataLoader, FvcomPlotConfig, FvcomPlotter
# from xfvcom.helpers import FrameGenerator
from xfvcom.plot_utils import create_anim_2d_plot
from xfvcom.plot_options import FvcomPlotOptions
import os
import pandas as pd
import numpy as np
# from functools import partial
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

### Prepare FvcomDataLoader instance of `fvcom` using FVCOM output netcdf.
- It will take time.
- Dataset is `fvcom.ds`.

In [ ]:
# Loading FVCOM output netcdf
# List of netcdf files convenient to switch to another netcdf by specifying its index
base_path = "~/Github/TB-FVCOM/goto2023/output"
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
#base_path =  "/home/pj24001722/ku40003295/Ersem_TokyoBay/output_5times"
#ncfiles = ["tb_0001.nc"]
base_path = os.path.expanduser(base_path)
index_ncfile = 0
ncfile_path = f"{base_path}/{ncfiles[index_ncfile]}"
# Create an instance of FvcomDataLoader where fvcom.ds is a Dataset
fvcom = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)

In [ ]:
fvcom.ds

### Additionally creating MP4 and setting `cleanup`
- generate_mp4 (bool): Whether creating MP4 animation from GIF animation.
- cleanup (bool): Whether each frame is removed after creating animation.

In [ ]:
# Switch GIF/MP4 and set `cleanup`
generate_gif = True
generate_mp4 = False
cleanup = False

### Create GIF and/or MP4 animation
- 2-D horizontal plot with customizing by updating `ax`, which changes with time.
- Prepare `custome_plot` for customizing or `post_process_func=None` without customizing.  

In [ ]:
def custom_plot(ax, da, time):
    """
    Plot the corresponding datetime at each frame

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.    
    """
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

time = slice(0, 72)
# Check the time slice is within the valid range.
time_size = fvcom.ds.sizes['time']
print(f"Time dimension size: {time_size}")
start_index = time.start if time.start is not None else 0
end_index = time.stop if time.stop is not None else time_size
if start_index < 0 or end_index > time_size:
    raise IndexError(f"Time slice is out of bounds. Valid range is [0, {time_size}), "
                     f"but got slice({start_index}, {end_index}).")

dataset = fvcom.ds.isel(time=time) # You may slice by the time index range. The whole range is `time=slice(0, None)`.
plotter = FvcomPlotter(dataset, FvcomPlotConfig(figsize=(6, 8)))
# Specify the number of maximum processes.
# 16 may be the optimum number.
processes = 16
# Specify var_name and siglay if any
#var_name = "O2_o"
var_name = "temp"
siglay = 0
# Set plot_kwargs for `ax.tricontourf(**kwargs)`
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet"}
#plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True}
plot_kwargs={}

# Invoke xfvcom.plot_utils.create_gif_anim_2d_plot
opts = FvcomPlotOptions.from_kwargs(**plot_kwargs)
create_anim_2d_plot(plotter, processes, var_name, siglay=siglay, fps=10,
                    generate_mp4=generate_mp4, cleanup=cleanup,
                    post_process_func=custom_plot, opts=opts)